In [8]:
class Permutation:
    """
    Store a permutation of the set {1,2,...,2n} using a dict:
       perm[x] = y
    means x is sent to y.
    """
    def __init__(self, mapping):
        # Copy the user's dictionary so we have our own internal storage
        self.perm = dict(mapping)

    def __call__(self, x):
        """Apply this permutation to an element x."""
        return self.perm[x]

    def __repr__(self):
        return f"Permutation({self.perm})"

In [9]:
p = Permutation({1: 2, 2: 3, 3: 1, 4: 4})
print(p)  # Permutation({1: 2, 2: 3, 3: 1, 4: 4})

Permutation({1: 2, 2: 3, 3: 1, 4: 4})


In [12]:
class RibbonGraphByPermutations:
    """
    An oriented ribbon graph D = (sigma0, sigma1, sigma2),
    each sigma is a permutation on {1, 2, ..., 2n}.
    """
    def __init__(self, sigma0, sigma1, sigma2):
        # Each sigma_i is a Permutation object or a dict
        self.sigma0 = sigma0
        self.sigma1 = sigma1
        self.sigma2 = sigma2

    def __repr__(self):
        return (f"RibbonGraphByPermutations(\n"
                f"  sigma0={self.sigma0},\n"
                f"  sigma1={self.sigma1},\n"
                f"  sigma2={self.sigma2}\n)")

In [15]:
#######################################################################
# 1) Data structures for permutations and the ribbon-graph triple
#######################################################################

class Permutation:
    """
    Store a permutation of {1, 2, ..., M} as a dict: perm[x] = y means x is sent to y.
    """
    def __init__(self, mapping=None):
        self.perm = dict(mapping) if mapping else {}

    def __call__(self, x):
        return self.perm[x]

    def __setitem__(self, x, y):
        self.perm[x] = y

    def items(self):
        return self.perm.items()

    def __repr__(self):
        return f"Permutation({self.perm})"


class RibbonGraphByPermutations:
    """
    Minimal class for a ribbon graph specified by three permutations (sigma0, sigma1, sigma2).
    """
    def __init__(self, sigma0, sigma1, sigma2):
        self.sigma0 = sigma0
        self.sigma1 = sigma1
        self.sigma2 = sigma2

    def __repr__(self):
        return (f"RibbonGraphByPermutations(\n"
                f"  sigma0={self.sigma0},\n"
                f"  sigma1={self.sigma1},\n"
                f"  sigma2={self.sigma2}\n)")


#######################################################################
# 2) “Potholder” -> Kauffman state arcs
#
#   We treat each crossing as having 4 “corners”: NW, NE, SE, SW.
#   A smoothing choice 0 or 1 decides how corners connect into arcs.
#######################################################################

def corner_id(i, j, corner):
    """
    Give a unique integer label to the corner of the crossing at row i, col j.
    corner is one of ('NW','NE','SE','SW').
    We’ll map each such triple to an integer that we can treat as “half-edge ID.”
    """
    # You can choose any encoding you like. For instance:
    #   ID = (i * n + j)*4 + offset_for_corner
    # where offset_for_corner is 0..3 for NW..SW.
    corner_map = {'N':0, 'S':1, 'E':2, 'W':3}
    base = (i*n + j)*4
    return base + corner_map[corner] + 1  # +1 so IDs start from 1, not 0

def smoothing_arcs(i, j, crossing_bit):
    """
    Return pairs of corners that form arcs at crossing (i,j),
    depending on crossing_bit (0 => A-smoothing, 1 => B-smoothing).
    """
    if crossing_bit == 0:
        # A-smoothing: N->W and S->E
        return [(corner_id(i,j,'N'), corner_id(i,j,'W')),
                (corner_id(i,j,'S'), corner_id(i,j,'E'))]
    else:
        # A-smoothing: N->E and S->W
        return [(corner_id(i,j,'N'), corner_id(i,j,'E')),
                (corner_id(i,j,'S'), corner_id(i,j,'Ws'))]


#######################################################################
# 3) Main construction: build sigma0, sigma1, sigma2 from an n x n
#    "potholder" described by a length-n^2 bitstring.
#######################################################################

def build_potholder_ribbon_graph(n, crossing_bits):
    """
    Given:
       n: odd integer number of horizontal/vertical strands
       crossing_bits: a string of length n^2 of '0'/'1' indicating A/B splicings

    Construct the triple (sigma0, sigma1, sigma2) as permutations in dict form,
    then wrap them in RibbonGraphByPermutations.
    """
    # 1) Gather all arcs from the local smoothings
    #    arcs_set will store pairs (startID, endID) describing which corners are joined
    arcs_set = []
    index = 0
    for i in range(n):
        for j in range(n):
            bit = int(crossing_bits[index])
            index += 1
            arcs_set.extend(smoothing_arcs(i, j, bit))

    # arcs_set is a list of edges, each is (cornerA, cornerB).
    # If we orient them, we get half-edges in both directions;
    # for Kauffman states, we usually orient the circles so that
    # each arc is directed. For now, we’ll just keep them as undirected pairs,
    # and define half-edges = corners. Then we'll define:
    #   sigma1: pairs the two corners that belong to the same crossing chord
    #   sigma0: the circular ordering around each loop (we must trace loops)
    #   sigma2: typically a “face” permutation from the standard definition

    # 2) Build sigma1 from arcs_set
    #    Each chord is (a,b), so a->b and b->a in sigma1.
    sigma1_map = {}
    for (a,b) in arcs_set:
        sigma1_map[a] = b
        sigma1_map[b] = a

    # 3) Figure out the loops to build sigma0.
    #    Each corner belongs to exactly one loop. We find these loops
    #    by “walking around the circle.” For a standard Kauffman state,
    #    that means going from corner -> next corner horizontally or vertically
    #    in the diagram, but to keep it short, we can just do a brute-force adjacency
    #    where we see which corners connect as “neighbors” around the circle.
    #
    #    A simpler approach: Each chord (a->b in sigma1) splits a circle in two arcs.
    #    If we want the oriented circle, we need to track how corners line up
    #    along the original strands. For an actual implementation, you’d need
    #    to attach each corner to its “next corner on the same circle.” That’s sigma0.

    # For demonstration, let's *pretend* each horizontal row or vertical column
    # lumps corners in a line. We'll do a naive adjacency. A real approach
    # must track geometry carefully! For now, we’ll just link each corner "a"
    # to "a+1" mod total if they appear on the same loop, etc.

    # We'll do a naive approach: sort all corner IDs, link them sequentially as if
    # each is the next in the circle. This won't match real geometry if n>1, but
    # it *does* illustrate how to fill sigma0 as a cycle permutation.

    # In real code, you'd trace out each loop explicitly: 
    #   - pick an unvisited corner c
    #   - follow it around the loop using the connectivity of the smoothing
    #   - record that cycle into sigma0
    all_corners = sorted(sigma1_map.keys())  # all corners we have
    sigma0_map = {}
    for i in range(len(all_corners)):
        a = all_corners[i]
        b = all_corners[(i+1)%len(all_corners)]
        sigma0_map[a] = b

    # 4) Build sigma2 (face permutation).
    #    Typically, sigma2 is found by taking each corner’s “dual edge” around the boundary
    #    of the embedded surface. For demonstration, we will just define sigma2 = identity:
    #    in real use, you would gather boundary cycles by walking around faces of the diagram.
    sigma2_map = {}
    for c in all_corners:
        sigma2_map[c] = c  # identity for demonstration

    # Convert these dicts into Permutation objects:
    sigma0 = Permutation(sigma0_map)
    sigma1 = Permutation(sigma1_map)
    sigma2 = Permutation(sigma2_map)

    return RibbonGraphByPermutations(sigma0, sigma1, sigma2)


#######################################################################
# 4) Example usage
#######################################################################

if __name__ == "__main__":
    n = 3
    # Suppose we define a 3x3 = 9-bit string of 0/1 randomly:
    bits = "010100111"
    RG = build_potholder_ribbon_graph(n, bits)
    print(RG)


KeyError: 'NW'

In [2]:
class Corner:
    """
    A Corner object is a triple (vertex, incoming, outgoing),
    where 'vertex' is the Vertex this corner lies on,
    and 'incoming', 'outgoing' are the indices of consecutive edges 
    (in the cyclic order at 'vertex').
    """
    def __init__(self, vertex, incoming, outgoing):
        self.vertex = vertex      # A reference to a Vertex object
        self.incoming = incoming  # Index of the incoming edge in vertex.edges
        self.outgoing = outgoing  # Index of the outgoing edge in vertex.edges

    def __repr__(self):
        return f"Corner(vertex={self.vertex}, incoming={self.incoming}, outgoing={self.outgoing})"


class BoundaryCycle:
    """
    A BoundaryCycle object is a collection (often a set or list) of Corner objects
    that form one boundary cycle of the ribbon graph.
    """
    def __init__(self, corners=None):
        # corners can be a list or set of Corner objects
        self.corners = corners if corners else []

    def __repr__(self):
        return f"BoundaryCycle(corners={self.corners})"


class Edge:
    """
    An Edge object is an unordered pair of endpoints, where each endpoint is 
    a (vertex, attachment_index) tuple. The 'attachment_index' is the index 
    within the vertex's edge list that corresponds to this Edge.
    """
    def __init__(self, endpoint1, endpoint2):
        # Each endpoint is a tuple (vertex, index)
        # 'vertex' is a Vertex object; 'index' is the position of this edge in vertex.edges
        self.endpoint1 = endpoint1  # (Vertex, index)
        self.endpoint2 = endpoint2  # (Vertex, index)

    def __repr__(self):
        return f"Edge(endpoint1={self.endpoint1}, endpoint2={self.endpoint2})"


class Vertex:
    """
    A Vertex object is a list of the labels (or references) to attached edges.
    These edges are stored in cyclic order, meaning that rotating this list
    should be considered the same vertex (for isomorphism purposes).
    """
    def __init__(self, edges=None):
        # edges is typically a list of edge labels or references to Edge objects
        # in a cyclic order
        self.edges = edges if edges else []

    def __repr__(self):
        return f"Vertex(edges={self.edges})"


class RibbonGraph:
    """
    A RibbonGraph object consists of:
      - A list of Vertex objects
      - A list of Edge objects
      - A collection (list/set) of BoundaryCycle objects
      - An orientation (often stored as a list that associates each edge with a position).
    """
    def __init__(self, vertices=None, edges=None, boundarycycles=None, orientation=None):
        self.vertices = vertices if vertices else []
        self.edges = edges if edges else []
        self.boundarycycles = boundarycycles if boundarycycles else []
        # Orientation is a list or another structure that captures 
        # the ordering/orientation of edges. Two lists differ by an even permutation 
        # can be considered equivalent.
        self.orientation = orientation if orientation else []

    def __repr__(self):
        return (
            f"RibbonGraph(\n"
            f"  vertices={self.vertices},\n"
            f"  edges={self.edges},\n"
            f"  boundarycycles={self.boundarycycles},\n"
            f"  orientation={self.orientation}\n"
            f")"
        )

In [4]:
# First, create two Vertex objects (these will later have references to the edges).
v_a = Vertex()
v_b = Vertex()

# Create three Edge objects. The tuple (vertex, index) tells each endpoint
# "this Edge is located at position 'index' in the vertex's edge list."
e0 = Edge((v_a, 0), (v_b, 0))
e1 = Edge((v_a, 1), (v_b, 1))
e2 = Edge((v_a, 2), (v_b, 2))

# Now fill in the edge lists for each vertex in the correct cyclic order:
# Vertex a has [e0, e1, e2]; vertex b has [e0, e2, e1].
v_a.edges = [e0, e1, e2]
v_b.edges = [e0, e2, e1]

# Next, define the corners that make up a boundary cycle:
# Here we have four corners: 
#   (a,0,1), (b,2,0), (a,1,2), (b,1,2)
# so we build them as Corner(vertex, incoming_edge_index, outgoing_edge_index).
c1 = Corner(v_a, 0, 1)
c2 = Corner(v_b, 2, 0)
c3 = Corner(v_a, 1, 2)
c4 = Corner(v_b, 1, 2)

# Collect these corners into a single BoundaryCycle.
bc = BoundaryCycle([c1, c2, c3, c4])

# We also specify an orientation for this graph—here, we can just store 
# the edge indices [0, 1, 2] corresponding to [e0, e1, e2].
orientation = [0, 1, 2]

# Finally, assemble the RibbonGraph:
G = RibbonGraph(
    vertices=[v_a, v_b],
    edges=[e0, e1, e2],
    boundarycycles=[bc],
    orientation=orientation
)

In [6]:
G.vertices

[Vertex(edges=[Edge(endpoint1=(Vertex(edges=[...]), 0), endpoint2=(Vertex(edges=[Edge(endpoint1=(Vertex(edges=[...]), 0), endpoint2=(...)), Edge(endpoint1=(Vertex(edges=[...]), 2), endpoint2=(Vertex(edges=[...]), 2)), Edge(endpoint1=(Vertex(edges=[...]), 1), endpoint2=(Vertex(edges=[...]), 1))]), 0)), Edge(endpoint1=(Vertex(edges=[...]), 1), endpoint2=(Vertex(edges=[Edge(endpoint1=(Vertex(edges=[...]), 0), endpoint2=(Vertex(edges=[...]), 0)), Edge(endpoint1=(Vertex(edges=[...]), 2), endpoint2=(Vertex(edges=[...]), 2)), Edge(endpoint1=(Vertex(edges=[...]), 1), endpoint2=(...))]), 1)), Edge(endpoint1=(Vertex(edges=[...]), 2), endpoint2=(Vertex(edges=[Edge(endpoint1=(Vertex(edges=[...]), 0), endpoint2=(Vertex(edges=[...]), 0)), Edge(endpoint1=(Vertex(edges=[...]), 2), endpoint2=(...)), Edge(endpoint1=(Vertex(edges=[...]), 1), endpoint2=(Vertex(edges=[...]), 1))]), 2))]),
 Vertex(edges=[Edge(endpoint1=(Vertex(edges=[Edge(endpoint1=(...), endpoint2=(Vertex(edges=[...]), 0)), Edge(endpoint1=